<a href="https://colab.research.google.com/github/GiuBonacchi/Music/blob/main/sistema_de_reconhecimento_musical_util.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Colocar dataframe com nome novos_dados.csv

In [471]:
import pickle
import pandas as pd
import numpy as np
from ast import literal_eval
from sklearn.preprocessing import StandardScaler, MultiLabelBinarizer

In [472]:
with open('modelo_random_forest.pkl', 'rb') as f:
    model = pickle.load(f)
with open('mlb.pkl', 'rb') as f:
    mlb = pickle.load(f)
with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)
with open('colunas_usadas_no_treino.pkl', 'rb') as f:
    colunas_treino = pickle.load(f)

In [473]:
novos_dados = pd.read_csv('dataframe_musicas.csv')
print(novos_dados.head())

                music         artist  ...  followers  liked
0                Hurt       NewJeans  ...   10899590      1
1             Gravity     John Mayer  ...    6181000      1
2  If I Ain't Got You    Alicia Keys  ...   12650472      1
3                Love   Keyshia Cole  ...    4224826      1
4                Home  Michael Bublé  ...    6873401      1

[5 rows x 8 columns]


In [474]:
def safe_literal_eval(val):
    try:
        return literal_eval(val) if isinstance(val, str) else []
    except (SyntaxError, ValueError):
        return []

novos_dados['genres'] = novos_dados['genres'].apply(safe_literal_eval)

In [475]:
genres_binarized = mlb.transform(novos_dados['genres'])
genres_binarized_df = pd.DataFrame(genres_binarized, columns=mlb.classes_)

In [476]:
music_artist = novos_dados[['music','artist']].reset_index(drop=True)

In [477]:
remover_colunas = [col for col in ['liked', 'music', 'artist',]if col in novos_dados.columns]
novos_dados = novos_dados.drop(columns=remover_colunas)

In [478]:
x_novo = pd.concat([x_novo.reset_index(drop=True), genres_binarized_df.reset_index(drop=True)], axis=1)

In [479]:
numeric_cols = ['duration_ms', 'music_popularity', 'artist_popularity', 'followers']

In [480]:
for cols in numeric_cols:
    if cols not in x_novo.columns:
        x_novo[cols] = 0

In [481]:
x_novo[numeric_cols] = scaler.transform(x_novo[numeric_cols])

In [482]:
for col in colunas_treino:
    if col not in x_novo.columns:
        x_novo[col] = 0

colunas_extras = [col for col in x_novo.columns if col not in colunas_treino]
if colunas_extras:
    x_novo = x_novo.drop(columns=colunas_extras)

In [483]:
x_novo = x_novo.loc[:,colunas_treino]

In [484]:
previsoes = model.predict(x_novo)

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- genres
Feature names seen at fit time, yet now missing:
- acid jazz
- acid rock
- acoustic pop
- alternative metal
- alternative rock
- ...


In [ ]:
previsoes_df = music_artist.copy()
previsoes_df['liked_predicted'] = previsoes

In [ ]:
print(previsoes_df.head())

In [ ]:
print(colunas_treino)
print(len(colunas_treino))

In [ ]:
print(list(x_novo.columns))
print(len(x_novo.columns))

In [ ]:
print("Colunas do modelo:")
for col in colunas_treino:
    print(repr(col))

print("Colunas do x_novo:")
for col in x_novo.columns:
    print(repr(col))